# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [55]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [56]:
next(iter(testloader))


[tensor([[[[ 1.4440,  1.4440,  1.4269,  ...,  1.9407,  1.8722,  1.4269],
           [ 1.4612,  1.4612,  1.4783,  ...,  1.7865,  1.8550,  1.8550],
           [ 1.4954,  1.4440,  1.4612,  ...,  1.1529,  1.8208,  1.7694],
           ...,
           [ 0.3823,  0.3994, -0.0972,  ...,  1.6495,  1.1015,  1.1358],
           [ 0.3652,  0.4851,  0.3823,  ...,  1.7523,  1.3413,  1.9064],
           [ 0.0741, -0.0287,  0.3309,  ...,  1.7180,  1.6838,  1.9749]],
 
          [[ 1.8158,  1.8508,  1.8683,  ...,  2.1310,  2.0609,  1.6057],
           [ 1.8158,  1.8158,  1.8333,  ...,  1.9734,  2.0434,  2.0434],
           [ 1.8333,  1.7808,  1.7983,  ...,  1.3431,  2.0084,  1.9559],
           ...,
           [ 0.6954,  0.7129,  0.2052,  ...,  1.7633,  1.1856,  1.2381],
           [ 0.6779,  0.8004,  0.6954,  ...,  1.8508,  1.4482,  2.0259],
           [ 0.3978,  0.2752,  0.6604,  ...,  1.8158,  1.7633,  2.0609]],
 
          [[ 2.0300,  2.0648,  2.0648,  ...,  2.4134,  2.3611,  1.9080],
           [ 

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 33782142.96it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [5]:
import time

In [6]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"DEVICE = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

DEVICE = cpu; Time per batch: 5.552 seconds
DEVICE = cuda; Time per batch: 0.009 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [7]:
# TODO: Train a model with a pre-trained network

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [8]:
epochs = 3
print_every = 40
steps = 0

# change to cuda
model.to('cuda')

for e in range(epochs):
    running_loss = 0
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every))
            
            running_loss = 0

Epoch: 1/3...  Loss: 0.2509
Epoch: 1/3...  Loss: 0.1873
Epoch: 1/3...  Loss: 0.1741
Epoch: 1/3...  Loss: 0.1861
Epoch: 1/3...  Loss: 0.1646
Epoch: 1/3...  Loss: 0.1461
Epoch: 1/3...  Loss: 0.1488
Epoch: 1/3...  Loss: 0.1648
Epoch: 2/3...  Loss: 0.0366
Epoch: 2/3...  Loss: 0.1493
Epoch: 2/3...  Loss: 0.1743
Epoch: 2/3...  Loss: 0.1269
Epoch: 2/3...  Loss: 0.1691
Epoch: 2/3...  Loss: 0.1506
Epoch: 2/3...  Loss: 0.1488
Epoch: 2/3...  Loss: 0.1247
Epoch: 2/3...  Loss: 0.1326
Epoch: 3/3...  Loss: 0.0548
Epoch: 3/3...  Loss: 0.1528
Epoch: 3/3...  Loss: 0.1379
Epoch: 3/3...  Loss: 0.1337
Epoch: 3/3...  Loss: 0.1360
Epoch: 3/3...  Loss: 0.1366
Epoch: 3/3...  Loss: 0.1280
Epoch: 3/3...  Loss: 0.1446
Epoch: 3/3...  Loss: 0.1391


tensor([[[[ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5639],
          [ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5468],
          [ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5639],
          ...,
          [-0.3198, -0.3027, -0.2684,  ...,  0.6734,  0.6392,  0.6734],
          [-0.2513, -0.2171, -0.1828,  ...,  0.7077,  0.6734,  0.6734],
          [-0.2171, -0.1486, -0.1657,  ...,  0.7762,  0.7419,  0.7077]],

         [[ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5532],
          [ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5357],
          [ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5532],
          ...,
          [-0.5651, -0.5301, -0.4951,  ...,  0.6078,  0.5728,  0.5903],
          [-0.4951, -0.4426, -0.3901,  ...,  0.5728,  0.5728,  0.5728],
          [-0.4426, -0.3725, -0.3725,  ...,  0.6078,  0.5903,  0.5903]],

         [[ 0.8971,  0.8797,  0.8797,  ...,  0.7054,  0.7054,  0.7054],
          [ 0.8971,  0.8797,  

In [57]:
correct = 0
total = 0
model.to('cpu')
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))



Accuracy of the network on the 10000 test images: 96 %


In [37]:
next(iter(testloader))
#torch.max(outputs.data, 1)

[tensor([[[[ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5639],
           [ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5468],
           [ 1.5468,  1.5639,  1.5639,  ...,  1.5639,  1.5639,  1.5639],
           ...,
           [-0.3198, -0.3027, -0.2684,  ...,  0.6734,  0.6392,  0.6734],
           [-0.2513, -0.2171, -0.1828,  ...,  0.7077,  0.6734,  0.6734],
           [-0.2171, -0.1486, -0.1657,  ...,  0.7762,  0.7419,  0.7077]],
 
          [[ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5532],
           [ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5357],
           [ 1.5357,  1.5532,  1.5532,  ...,  1.5532,  1.5532,  1.5532],
           ...,
           [-0.5651, -0.5301, -0.4951,  ...,  0.6078,  0.5728,  0.5903],
           [-0.4951, -0.4426, -0.3901,  ...,  0.5728,  0.5728,  0.5728],
           [-0.4426, -0.3725, -0.3725,  ...,  0.6078,  0.5903,  0.5903]],
 
          [[ 0.8971,  0.8797,  0.8797,  ...,  0.7054,  0.7054,  0.7054],
           [ 

In [ ]:
# Putting the above into functions, so they can be used later

def do_deep_learning(model, trainloader, epochs, print_every, criterion, optimizer, device='cpu'):
    epochs = epochs
    print_every = print_every
    steps = 0

    # change to cuda
    model.to('cuda')

    for e in range(epochs):
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1

            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every))

                running_loss = 0
    
def check_accuracy_on_test(testloader):    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
    
do_deep_learning(model, trainloader, 3, 40, criterion, optimizer, 'gpu')
check_accuracy_on_test(testloader)